# Segmenting and Clustering Neighborhoods in Toronto

_hefangzhang  
Apr 7, 2020_

In [5]:
# import
import numpy as np
import pandas as pd
import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


## Step1: Load and transform the data

In [44]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source)
w_data=soup.find('div',class_='mw-parser-output')
t_data=w_data.table.tbody

In [45]:
columns=['PostalCode','Borough','Neighbourhood']
data=dict({key:[]*len(columns) for key in columns})

for row in t_data.find_all('tr'):
    for v,column in zip(row.find_all('td'),columns):
        v=v.text
        v=v.replace('\n','')
        data[column].append(v)

In [46]:
df=pd.DataFrame.from_dict(data=data)[columns]
print(df.shape)
print(df.dtypes)
df.head()

(180, 3)
PostalCode       object
Borough          object
Neighbourhood    object
dtype: object


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


**Ignore cells with a borough that is Not assigned.**

In [47]:
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.shape

(103, 3)

**
Combined into one row with the neighborhoods separated with a comma
There is actually no More than one neighborhood exist in one postal code area
But show the method here
**

In [40]:
# Use PostalCode, Borough as the key

postcodes=df['PostalCode'].values
boroughs=df['Borough'].values
neighbours=df['Neighbourhood'].values

dic=dict({(key1,key2):[] for key1,key2 in zip(postcodes,boroughs)})
for postcode,borough,neighbour in zip(postcodes,boroughs,neighbours):
    key=(postcode,borough)
    dic[key].append(neighbour)
    
df=pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])
for key,value in dic.items():
    postcode,borough,neighbour=key[0],key[1],value
    neighbour=','.join(neighbour)
    df=df.append({'PostalCode':postcode,
                  'Borough':borough,
                  'Neighbourhood':neighbour},ignore_index=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [52]:
# Use the .shape method to print the number of rows of the dataframe
df.shape

(103, 3)